In [1]:
"""
INTEGRATED: EXPERIMENT 2 → EXPERIMENT 3 PIPELINE
================================================

This script runs:
1. EXPERIMENT 2: Train adversarial classifier with hard negative mining
2. EXPERIMENT 3: Apply contrastive learning on logical pairs using the trained model

Your friend can run this entire pipeline in one go!

Usage:
    python integrated_exp2_exp3.py

Estimated time: 3-4 hours on CPU, 1-1.5 hours on GPU
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
from transformers import (
    DistilBertModel,
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    get_linear_schedule_with_warmup
)
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import json
from collections import defaultdict
from pathlib import Path
# Set random seeds
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [2]:
# ============================================================================
# EXPERIMENT 2 COMPONENTS (From your notebook)
# ============================================================================

class LogicalSentenceDetector:
    """Enhanced heuristic-based detector for logical sentences"""

    def __init__(self):
        self.conditional_words = [
            'if', 'then', 'when', 'whenever', 'unless',
            'provided that', 'as long as', 'in case', 'supposing',
            'assuming', 'given that', 'on condition that'
        ]
        self.causal_words = [
            'because', 'since', 'as', 'due to', 'owing to',
            'therefore', 'thus', 'hence', 'consequently', 'so',
            'for this reason', 'as a result', 'leads to', 'causes'
        ]
        self.contrast_words = [
            'although', 'though', 'however', 'but', 'yet',
            'nevertheless', 'nonetheless', 'despite', 'in spite of',
            'whereas', 'while', 'on the other hand', 'conversely'
        ]
        self.conjunction_words = ['and', 'or', 'nor', 'either', 'neither']

        self.all_logic_words = (
            self.conditional_words + self.causal_words +
            self.contrast_words + self.conjunction_words
        )

    def compute_logic_score(self, sentence):
        """Compute a confidence score for logical structure (0-1)"""
        sentence_lower = sentence.lower()
        score = 0.0

        connective_count = sum(1 for word in self.all_logic_words if word in sentence_lower)
        score += min(0.4, connective_count * 0.2)

        if ',' in sentence or ';' in sentence:
            score += 0.3

        clause_count = sentence.count(',') + sentence.count(';')
        score += min(0.2, clause_count * 0.1)

        if re.search(r'if .+, .+', sentence_lower):
            score += 0.1
        elif re.search(r'because .+, .+', sentence_lower):
            score += 0.1

        return min(1.0, score)

    def is_logical(self, sentence, threshold=0.5):
        return self.compute_logic_score(sentence) >= threshold

    def get_logic_type(self, sentence):
        sentence_lower = sentence.lower()

        if any(word in sentence_lower for word in self.conditional_words):
            return 'conditional'
        elif any(word in sentence_lower for word in self.causal_words):
            return 'causal'
        elif any(word in sentence_lower for word in self.contrast_words):
            return 'contrast'
        elif any(word in sentence_lower for word in self.conjunction_words):
            return 'conjunction'
        return 'none'

In [3]:
class AdversarialGenerator:
    """Generate adversarial examples through multiple perturbation strategies"""

    def __init__(self):
        self.negations = ['not', 'never', "n't"]

        self.connective_synonyms = {
            'if': ['when', 'whenever', 'in case', 'supposing'],
            'because': ['since', 'as', 'due to the fact that'],
            'although': ['though', 'even though', 'despite the fact that'],
            'therefore': ['thus', 'hence', 'consequently', 'as a result'],
            'however': ['nevertheless', 'nonetheless', 'yet', 'but'],
            'when': ['whenever', 'as', 'while'],
        }

        self.opposing_connectives = {
            'because': ['although', 'despite', 'even though'],
            'although': ['because', 'since'],
            'if': ['even if', 'unless'],
            'therefore': ['however', 'nevertheless'],
            'and': ['but', 'yet'],
        }

        self.verb_synonyms = {
            'studied': ['learned', 'reviewed', 'practiced'],
            'passed': ['succeeded', 'completed', 'aced'],
            'rains': ['pours', 'drizzles', 'showers'],
            'rises': ['comes up', 'appears', 'ascends'],
            'falls': ['drops', 'decreases', 'declines'],
            'grows': ['develops', 'expands', 'increases'],
        }

    def add_negation(self, sentence):
        verbs = ['is', 'are', 'was', 'were', 'will', 'would', 'can', 'could', 'should', 'may', 'might']
        for verb in verbs:
            pattern = rf'\b{verb}\b\s+'
            if re.search(pattern, sentence, re.IGNORECASE):
                return re.sub(pattern, f'{verb} not ', sentence, count=1, flags=re.IGNORECASE)

        if ',' in sentence:
            parts = sentence.split(',', 1)
            if len(parts) == 2:
                return parts[0] + ', not' + parts[1]
        return sentence

    def remove_negation(self, sentence):
        sentence = re.sub(r'\bnot\b\s*', '', sentence, flags=re.IGNORECASE)
        sentence = re.sub(r'\bnever\b', 'always', sentence, flags=re.IGNORECASE)
        sentence = re.sub(r"n't\b", '', sentence)
        return sentence

    def flip_connective(self, sentence):
        sentence_lower = sentence.lower()

        for original, opposites in self.opposing_connectives.items():
            if original in sentence_lower:
                replacement = random.choice(opposites)
                return re.sub(rf'\b{original}\b', replacement, sentence, count=1, flags=re.IGNORECASE)

        return sentence

    def remove_connective(self, sentence):
        connectives = [
            'if', 'because', 'since', 'although', 'when', 'though',
            'therefore', 'thus', 'hence', 'so', 'but', 'however'
        ]
        for conn in connectives:
            pattern = rf'\b{conn}\b[,\s]*'
            result = re.sub(pattern, '', sentence, flags=re.IGNORECASE, count=1)
            if result != sentence:
                return result
        return sentence

    def swap_clauses(self, sentence):
        if ',' in sentence:
            parts = sentence.split(',', 1)
            if len(parts) == 2:
                clause1 = parts[0].strip()
                clause2 = parts[1].strip()
                clause2 = clause2[0].upper() + clause2[1:] if clause2 else clause2
                clause1 = clause1[0].lower() + clause1[1:] if clause1 else clause1
                return f"{clause2}, {clause1}"
        return sentence

    def synonym_replacement(self, sentence):
        sentence_lower = sentence.lower()

        for original, synonyms in self.connective_synonyms.items():
            if original in sentence_lower:
                replacement = random.choice(synonyms)
                return re.sub(rf'\b{original}\b', replacement, sentence, count=1, flags=re.IGNORECASE)

        return sentence

    def paraphrase_clause(self, sentence):
        for original, synonyms in self.verb_synonyms.items():
            if original in sentence.lower():
                replacement = random.choice(synonyms)
                sentence = re.sub(rf'\b{original}\b', replacement, sentence, count=1, flags=re.IGNORECASE)
                break
        return sentence

    def weaken_logic(self, sentence):
        hedges = ['probably', 'possibly', 'might', 'perhaps', 'usually']
        hedge = random.choice(hedges)

        verbs = ['is', 'are', 'will', 'would', 'can']
        for verb in verbs:
            pattern = rf'\b{verb}\b\s+'
            if re.search(pattern, sentence, re.IGNORECASE):
                return re.sub(pattern, f'{verb} {hedge} ', sentence, count=1, flags=re.IGNORECASE)

        return sentence

    def generate_perturbation(self, sentence, perturbation_type='random'):
        if perturbation_type == 'random':
            perturbation_type = random.choice([
                'add_negation', 'remove_negation', 'flip_connective',
                'remove_connective', 'swap_clauses', 'synonym_replacement',
                'paraphrase_clause', 'weaken_logic'
            ])

        try:
            if perturbation_type == 'add_negation':
                return self.add_negation(sentence)
            elif perturbation_type == 'remove_negation':
                return self.remove_negation(sentence)
            elif perturbation_type == 'flip_connective':
                return self.flip_connective(sentence)
            elif perturbation_type == 'remove_connective':
                return self.remove_connective(sentence)
            elif perturbation_type == 'swap_clauses':
                return self.swap_clauses(sentence)
            elif perturbation_type == 'synonym_replacement':
                return self.synonym_replacement(sentence)
            elif perturbation_type == 'paraphrase_clause':
                return self.paraphrase_clause(sentence)
            elif perturbation_type == 'weaken_logic':
                return self.weaken_logic(sentence)
        except:
            return sentence

        return sentence

    def generate_multi_perturbation(self, sentence, num_perturbations=2):
        result = sentence
        perturbation_types = [
            'add_negation', 'flip_connective', 'remove_connective',
            'swap_clauses', 'weaken_logic'
        ]

        selected_types = random.sample(perturbation_types, min(num_perturbations, len(perturbation_types)))

        for ptype in selected_types:
            result = self.generate_perturbation(result, ptype)
            if result == sentence:
                result = self.generate_perturbation(result, 'random')

        return result

In [4]:
def create_expanded_dataset():
    """Create comprehensive dataset with 300+ sentences (from your notebook)"""

    logical_sentences = [
        # Your exact sentences from the notebook
        "If it rains, the ground will be wet.",
        "If you heat water to 100°C, it boils.",
        "If the battery dies, the phone won't work.",
        "If you save money, you can buy what you want.",
        "When the sun sets, the temperature drops.",
        "When plants get sunlight, they grow faster.",
        "When you exercise regularly, your health improves.",
        "When the alarm rings, people evacuate the building.",
        "When prices rise, demand typically falls.",
        "When ice melts, it becomes water.",
        "If you touch fire, you get burned.",
        "If the door is locked, you cannot enter.",
        "If she calls, please let me know.",
        "When winter comes, birds migrate south.",
        "When the light turns red, cars must stop.",
        "If you don't water plants, they will die.",
        "When the temperature drops below zero, water freezes.",
        "If you study hard, you'll likely succeed.",
        "When the movie ends, the lights come on.",
        "If the wifi is down, we can't work online.",
        "When you mix blue and yellow, you get green.",
        "If the train is delayed, we'll miss our connection.",
        "When the battery is full, the charging stops.",
        "If you break the rules, there are consequences.",
        "When the season changes, fashion trends shift.",
        "If the economy grows, employment increases.",
        "When the sun shines, solar panels generate power.",
        "If you forget the password, you can't log in.",
        "When the timer beeps, the food is ready.",
        "If the signal is weak, calls drop frequently.",
        "Because he studied hard, he passed the exam.",
        "Since the store was closed, we went home.",
        "Because the road was icy, traffic moved slowly.",
        "Since he missed the bus, he arrived late.",
        "Because the bridge collapsed, the road was closed.",
        "Because the weather was nice, we went to the park.",
        "Since the evidence was clear, the jury convicted him.",
        "Because she was sick, she stayed home from work.",
        "Since the project was urgent, they worked overtime.",
        "Because prices increased, sales declined.",
        "Since the restaurant was full, we waited outside.",
        "Because the water was contaminated, people got sick.",
        "Since he forgot his keys, he couldn't enter.",
        "Because the storm was severe, flights were cancelled.",
        "Since the deadline passed, submissions were closed.",
        "Because the team practiced daily, they won the championship.",
        "Since the equipment broke, production stopped.",
        "Because the film was popular, tickets sold out quickly.",
        "Since she had experience, she got the job.",
        "Because the road was blocked, we took a detour.",
        "Therefore, we must act quickly to solve this problem.",
        "Thus, the hypothesis was proven correct.",
        "Hence, the company decided to expand operations.",
        "Consequently, many people lost their jobs.",
        "As a result, the ecosystem was severely damaged.",
        "Therefore, further research is necessary.",
        "Thus, the treaty was signed by all parties.",
        "Hence, the policy was changed immediately.",
        "Consequently, sales increased by thirty percent.",
        "As a result, the building had to be demolished.",
        "Although she was tired, she finished the project.",
        "Although it was expensive, they bought the car.",
        "Although the task was difficult, she succeeded.",
        "Despite the rain, the game continued.",
        "Though he was young, he was very wise.",
        "Although they lost, they played their best.",
        "Despite the warning, he proceeded anyway.",
        "Though it was late, they kept working.",
        "Although the evidence was weak, they prosecuted.",
        "Despite the risks, she took the job.",
        "Although the road was long, they kept walking.",
        "Though the odds were against them, they won.",
        "Despite the cost, quality is worth it.",
        "Although it was crowded, we found seats.",
        "Though he apologized, she remained angry.",
        "Despite the delay, we arrived on time.",
        "Although the recipe was complex, she made it perfectly.",
        "Though the exam was hard, most students passed.",
        "Despite the competition, our product succeeded.",
        "Although he was injured, he finished the race.",
        "However, the results were not what we expected.",
        "Nevertheless, the plan moved forward.",
        "The weather was bad, but the event proceeded anyway.",
        "She was exhausted, yet she continued working.",
        "The task was daunting, however they persevered.",
        "It was expensive, but the quality justified the price.",
        "He was nervous, yet he delivered an excellent speech.",
        "The journey was long, nevertheless they enjoyed it.",
        "The situation was dire, but hope remained.",
        "The odds were slim, yet they took the chance.",
        "If you water the plants and give them sunlight, they will flourish.",
        "Because the temperature dropped and the roads were icy, schools closed.",
        "Although she studied hard and prepared well, she was still nervous.",
        "When the alarm sounds and smoke is detected, evacuate immediately.",
        "If you combine effort with strategy, success becomes likely.",
        "Since the data was analyzed and patterns emerged, conclusions were drawn.",
        "Although the plan was risky and resources were limited, they proceeded.",
        "When technology advances and costs decrease, adoption increases.",
        "If ingredients are fresh and preparation is careful, meals taste better.",
        "Because demand was high and supply was low, prices soared.",
        "When citizens vote and participate actively, democracy thrives.",
        "If you listen carefully and ask questions, you learn more.",
        "Since the evidence was overwhelming and witnesses testified, the verdict was guilty.",
        "Although the journey was difficult and setbacks occurred, they reached their goal.",
        "When interest rates fall and borrowing becomes cheaper, economies grow.",
        "If systems are tested and bugs are fixed, software becomes reliable.",
        "Because training was thorough and equipment was modern, performance improved.",
        "When communication is clear and expectations are set, teams succeed.",
        "If you save consistently and invest wisely, wealth accumulates.",
        "Since regulations were strict and enforcement was strong, compliance improved.",
        "Unless you hurry, you will miss the train.",
        "Provided that you complete the assignment, you will pass.",
        "As long as you follow the rules, there won't be problems.",
        "In case of emergency, break the glass.",
        "Whenever she visits, she brings gifts.",
        "Since morning, the situation has deteriorated.",
        "Given that the facts support it, we should proceed.",
        "Assuming the weather holds, the picnic is on.",
        "On condition that you agree, we can move forward.",
        "Whereas some prefer coffee, others choose tea.",
        "While technology helps productivity, it also creates distractions.",
        "Either you adapt to change, or you get left behind.",
        "Neither rain nor snow will stop the delivery.",
        "Not only did she win, but she also set a record.",
        "Both the theory and the evidence support this conclusion.",
        "Just as the sun rises in the east, it sets in the west.",
        "The more you practice, the better you become.",
        "The harder you work, the luckier you get.",
        "No sooner had she left than the phone rang.",
        "Hardly had the game begun when it started raining.",
    ]

    non_logical_sentences = [
        # Your exact sentences from the notebook
        "The cat sleeps on the couch.",
        "She loves chocolate ice cream.",
        "The building is very tall.",
        "He drives a blue car.",
        "The movie was entertaining.",
        "They live in a small town.",
        "The coffee tastes bitter.",
        "She has three siblings.",
        "The book is on the table.",
        "He plays guitar every day.",
        "The flowers smell wonderful.",
        "She graduated last year.",
        "The museum opens at nine.",
        "He enjoys reading novels.",
        "The stars are bright tonight.",
        "She works as a teacher.",
        "The pizza was delicious.",
        "He speaks three languages.",
        "The concert starts soon.",
        "She painted the room yellow.",
        "The sky is blue today.",
        "Birds are singing in the trees.",
        "The ocean looks calm.",
        "Mountains surround the valley.",
        "Children are playing in the park.",
        "The restaurant serves Italian food.",
        "Her dress is red and elegant.",
        "The clock shows three o'clock.",
        "Music fills the air.",
        "The garden needs watering.",
        "His smile is contagious.",
        "The painting depicts a landscape.",
        "The library has many books.",
        "Her voice is melodious.",
        "The car needs repairs.",
        "The sunset is beautiful.",
        "The room is spacious and bright.",
        "The puppy is adorable.",
        "The water is crystal clear.",
        "The cake looks appetizing.",
        "She walked to the store yesterday.",
        "They watched a movie last night.",
        "He cooked dinner for his family.",
        "The team celebrated their victory.",
        "She writes in her journal daily.",
        "They traveled across Europe.",
        "He runs five miles every morning.",
        "The kids built a sandcastle.",
        "She organized the files carefully.",
        "They planted flowers in the garden.",
        "He repaired the broken fence.",
        "The artist created a masterpiece.",
        "She baked cookies for the party.",
        "They explored the ancient ruins.",
        "He learned to play the piano.",
        "The children sang songs together.",
        "She knitted a warm scarf.",
        "They renovated the old house.",
        "He photographed the wildlife.",
        "The dancers performed gracefully.",
        "The water is cold.",
        "The fabric feels soft.",
        "The room smells fresh.",
        "The music sounds peaceful.",
        "The surface is smooth.",
        "The night is dark.",
        "The air feels humid.",
        "The bread tastes stale.",
        "The light is dim.",
        "The ground is uneven.",
        "The atmosphere is tense.",
        "The mood is cheerful.",
        "The texture is rough.",
        "The temperature is moderate.",
        "The pressure is intense.",
        "The pace is slow.",
        "The style is modern.",
        "The tone is friendly.",
        "The flavor is spicy.",
        "The color is vibrant.",
        "I prefer tea over coffee.",
        "She thinks the movie is overrated.",
        "He believes in hard work.",
        "They enjoy outdoor activities.",
        "She appreciates good art.",
        "He values honesty.",
        "They admire courage.",
        "She finds mathematics interesting.",
        "He considers himself fortunate.",
        "They regard it as important.",
        "She loves classical music.",
        "He likes spicy food.",
        "They treasure old photographs.",
        "She cherishes her memories.",
        "He respects different opinions.",
        "They favor sustainable practices.",
        "She prefers quiet evenings.",
        "He enjoys intellectual discussions.",
        "They appreciate fine dining.",
        "She adores her grandchildren.",
        "The phone is ringing loudly.",
        "The traffic is heavy this morning.",
        "The leaves are changing colors.",
        "The baby is sleeping peacefully.",
        "The crowd cheered enthusiastically.",
        "The engine is making strange noises.",
        "The project deadline is approaching.",
        "The students are taking notes.",
        "The wind is blowing strongly.",
        "The fireplace is burning brightly.",
        "The snow is falling gently.",
        "The audience applauded warmly.",
        "The bread is baking in the oven.",
        "The river flows through the city.",
        "The clock is ticking quietly.",
        "The fog is rolling in.",
        "The champagne is chilling.",
        "The documents are being printed.",
        "The elevator is descending.",
        "The candles are flickering softly.",
        "His handwriting is illegible.",
        "The software is updating automatically.",
        "The news is spreading quickly.",
        "The battery is draining fast.",
        "The crowd is dispersing gradually.",
        "The ice is melting slowly.",
        "The prices are fluctuating daily.",
        "The negotiations are ongoing.",
        "The situation is improving steadily.",
        "The performance exceeded expectations.",
        "The product received positive reviews.",
        "The conference attracted many attendees.",
        "The curriculum includes various subjects.",
        "The menu offers vegetarian options.",
        "The neighborhood is quiet and safe.",
        "The festival features local artists.",
        "The collection showcases contemporary art.",
        "The documentary explores historical events.",
        "The workshop teaches practical skills.",
        "The campaign raised significant funds.",
    ]

    data = []
    for sent in logical_sentences:
        data.append({'sentence': sent, 'label': 1, 'is_perturbed': False, 'source': 'original'})
    for sent in non_logical_sentences:
        data.append({'sentence': sent, 'label': 0, 'is_perturbed': False, 'source': 'original'})

    print(f"Created base dataset: {len(logical_sentences)} logical, {len(non_logical_sentences)} non-logical")
    return data, logical_sentences  # Return logical sentences for Experiment 3


def augment_with_multi_perturbations(data, generator, rounds=[1, 2, 3], aug_per_round=1.0):
    """Augment dataset with multiple rounds of perturbations"""

    augmented_data = data.copy()
    logical_sentences = [d for d in data if d['label'] == 1]
    perturbed_map = defaultdict(list)  # Track perturbations for Experiment 3

    print(f"\nGenerating adversarial examples with {len(rounds)} perturbation rounds...")

    for num_perturbs in rounds:
        num_augmentations = int(len(logical_sentences) * aug_per_round)

        for _ in range(num_augmentations):
            original = random.choice(logical_sentences)
            perturbed_sent = generator.generate_multi_perturbation(
                original['sentence'],
                num_perturbations=num_perturbs
            )

            if perturbed_sent != original['sentence']:
                augmented_data.append({
                    'sentence': perturbed_sent,
                    'label': 0,
                    'is_perturbed': True,
                    'source': f'perturb_{num_perturbs}x'
                })
                # Track for Experiment 3
                perturbed_map[original['sentence']].append(perturbed_sent)

        print(f"  Round {num_perturbs}: Generated {num_augmentations} examples")

    return augmented_data, perturbed_map

In [5]:
class HardNegativeMiner:
    """Mine hard negative examples that the model gets wrong"""

    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.hard_negatives = []

    def find_hard_negatives(self, data, threshold=0.3):
        self.model.eval()
        hard_examples = []

        with torch.no_grad():
            for item in tqdm(data, desc="Mining hard negatives"):
                inputs = self.tokenizer(
                    item['sentence'],
                    return_tensors='pt',
                    padding=True,
                    truncation=True,
                    max_length=128
                ).to(self.device)

                outputs = self.model(**inputs)
                probs = torch.softmax(outputs.logits, dim=1)
                pred = torch.argmax(probs, dim=1).item()
                confidence = probs[0][pred].item()

                is_wrong = (pred != item['label'])
                is_uncertain = (confidence < 0.7)

                if is_wrong or is_uncertain:
                    hard_examples.append({
                        **item,
                        'confidence': confidence,
                        'predicted': pred,
                        'is_hard': True
                    })

        print(f"Found {len(hard_examples)} hard negative examples")
        return hard_examples

    def augment_hard_negatives(self, hard_examples, generator, multiplier=2):
        augmented = []

        for item in hard_examples:
            for _ in range(multiplier):
                perturbed = generator.generate_multi_perturbation(
                    item['sentence'],
                    num_perturbations=random.randint(1, 3)
                )

                if perturbed != item['sentence']:
                    augmented.append({
                        'sentence': perturbed,
                        'label': 0,
                        'is_perturbed': True,
                        'source': 'hard_negative_aug'
                    })

        print(f"Generated {len(augmented)} augmentations from hard negatives")
        return augmented

In [6]:
class EnsembleClassifier:
    """Combine neural model with heuristic detector"""

    def __init__(self, model, tokenizer, detector, device, neural_weight=0.7):
        self.model = model
        self.tokenizer = tokenizer
        self.detector = detector
        self.device = device
        self.neural_weight = neural_weight
        self.heuristic_weight = 1.0 - neural_weight

    def predict(self, sentence):
        self.model.eval()

        with torch.no_grad():
            inputs = self.tokenizer(
                sentence,
                return_tensors='pt',
                padding=True,
                truncation=True,
                max_length=128
            ).to(self.device)

            outputs = self.model(**inputs)
            neural_probs = torch.softmax(outputs.logits, dim=1)[0]
            neural_score = neural_probs[1].item()

        heuristic_score = self.detector.compute_logic_score(sentence)

        ensemble_score = (self.neural_weight * neural_score +
                         self.heuristic_weight * heuristic_score)

        prediction = 1 if ensemble_score >= 0.5 else 0

        return {
            'prediction': prediction,
            'ensemble_score': ensemble_score,
            'neural_score': neural_score,
            'heuristic_score': heuristic_score
        }

In [7]:
class LogicDataset(Dataset):
    """PyTorch Dataset for logical sentence classification"""

    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        encoding = self.tokenizer(
            item['sentence'],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(item['label'], dtype=torch.long),
            'sentence': item['sentence']
        }


def train_epoch(model, dataloader, optimizer, scheduler, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in tqdm(dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    return total_loss / len(dataloader), correct / total


def evaluate(model, dataloader, device):
    """Evaluate the model"""
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return total_loss / len(dataloader), correct / total

In [8]:
# ============================================================================
# EXPERIMENT 3 COMPONENTS (Contrastive Learning)
# ============================================================================

class LogicalPairConstructor:
    """Constructs positive and negative logical pairs"""

    def __init__(self):
        self.paraphrase_templates = {
            'conditional': [
                ('if {premise}, {conclusion}', 'when {premise}, {conclusion}'),
                ('if {premise}, {conclusion}', '{conclusion} when {premise}'),
                ('if {premise}, then {conclusion}', '{conclusion} if {premise}'),
            ],
            'causal': [
                ('because {premise}, {conclusion}', '{conclusion} because {premise}'),
                ('since {premise}, {conclusion}', '{conclusion} since {premise}'),
                ('{conclusion} because {premise}', '{premise} causes {conclusion}'),
                ('because {premise}, {conclusion}', '{premise} leads to {conclusion}'),
            ]
        }

    def extract_premise_conclusion(self, sentence):
        """Extract premise and conclusion from logical sentence"""
        sentence_lower = sentence.lower()

        # Conditional patterns
        if_match = re.search(r'if\s+(.+?),\s*(.+)', sentence_lower)
        if if_match:
            return {
                'premise': if_match.group(1).strip(),
                'conclusion': if_match.group(2).strip(),
                'type': 'conditional',
                'original': sentence
            }

        when_match = re.search(r'when\s+(.+?),\s*(.+)', sentence_lower)
        if when_match:
            return {
                'premise': when_match.group(1).strip(),
                'conclusion': when_match.group(2).strip(),
                'type': 'conditional',
                'original': sentence
            }

        # Causal patterns
        because_match = re.search(r'because\s+(.+?),\s*(.+)', sentence_lower)
        if because_match:
            return {
                'premise': because_match.group(1).strip(),
                'conclusion': because_match.group(2).strip(),
                'type': 'causal',
                'original': sentence
            }

        since_match = re.search(r'since\s+(.+?),\s*(.+)', sentence_lower)
        if since_match:
            return {
                'premise': since_match.group(1).strip(),
                'conclusion': since_match.group(2).strip(),
                'type': 'causal',
                'original': sentence
            }

        therefore_match = re.search(r'(.+?),\s*(?:therefore|thus|hence)\s+(.+)', sentence_lower)
        if therefore_match:
            return {
                'premise': therefore_match.group(1).strip(),
                'conclusion': therefore_match.group(2).strip(),
                'type': 'causal',
                'original': sentence
            }

        return None

    def create_positive_pair(self, sentence):
        """Create positive pair: original + paraphrase"""
        parsed = self.extract_premise_conclusion(sentence)
        if not parsed:
            return None

        premise = parsed['premise']
        conclusion = parsed['conclusion']
        logic_type = parsed['type']

        if logic_type in self.paraphrase_templates:
            templates = self.paraphrase_templates[logic_type]
            template_pair = random.choice(templates)

            paraphrase = template_pair[1].format(premise=premise, conclusion=conclusion)
            paraphrase = paraphrase[0].upper() + paraphrase[1:] + '.'

            return {
                'sentence1': sentence,
                'sentence2': paraphrase,
                'label': 1,
                'type': 'paraphrase',
                'logic_type': logic_type
            }

        return None

    def create_negative_from_perturbation(self, original, perturbed):
        """Create negative pair from adversarial perturbation"""
        return {
            'sentence1': original,
            'sentence2': perturbed,
            'label': 0,
            'type': 'adversarial_negative',
            'logic_type': 'broken'
        }

    def create_random_negative(self, sentence, all_sentences):
        """Create negative pair: random unrelated sentences"""
        other = random.choice([s for s in all_sentences if s != sentence])
        return {
            'sentence1': sentence,
            'sentence2': other,
            'label': 0,
            'type': 'random_negative',
            'logic_type': 'unrelated'
        }


def construct_contrastive_dataset(logical_sentences, perturbed_map):
    """Construct contrastive dataset with positive and negative pairs"""

    constructor = LogicalPairConstructor()
    pairs = []

    print("\n" + "="*80)
    print("CONSTRUCTING CONTRASTIVE PAIRS FOR EXPERIMENT 3")
    print("="*80)
    # 1. Create positive pairs (paraphrases)
    print("\n[1/3] Creating positive pairs (paraphrases)...")
    positive_count = 0
    for sentence in tqdm(logical_sentences):
        positive_pair = constructor.create_positive_pair(sentence)
        if positive_pair:
            pairs.append(positive_pair)
            positive_count += 1

    print(f"✓ Created {positive_count} positive pairs")

    # 2. Create negative pairs from adversarial perturbations
    print("\n[2/3] Creating negative pairs (adversarial perturbations)...")
    adversarial_count = 0
    for original, perturbed_list in perturbed_map.items():
        for perturbed in perturbed_list[:2]:  # Top 2 perturbations per sentence
            neg_pair = constructor.create_negative_from_perturbation(original, perturbed)
            pairs.append(neg_pair)
            adversarial_count += 1

    print(f"✓ Created {adversarial_count} adversarial negative pairs")

    # 3. Create random negative pairs
    print("\n[3/3] Creating negative pairs (random unrelated)...")
    random_count = 0
    for sentence in logical_sentences:
        for _ in range(2):  # 2 random negatives per sentence
            random_neg = constructor.create_random_negative(sentence, logical_sentences)
            pairs.append(random_neg)
            random_count += 1

    print(f"✓ Created {random_count} random negative pairs")

    # Shuffle
    random.shuffle(pairs)

    # Statistics
    positive_total = sum(1 for p in pairs if p['label'] == 1)
    negative_total = sum(1 for p in pairs if p['label'] == 0)

    print("\n" + "="*80)
    print("Contrastive Dataset Statistics:")
    print(f"  Total pairs: {len(pairs)}")
    print(f"  Positive pairs: {positive_total}")
    print(f"  Negative pairs: {negative_total}")
    print(f"  Balance: {positive_total/len(pairs):.2%} positive")
    print("="*80)

    return pairs

In [9]:
class ContrastiveWrapper(nn.Module):
    """Wraps Experiment 2 classifier for contrastive learning"""

    def __init__(self, phase2_model, temperature=0.05, freeze_base=False):
        super().__init__()

        self.encoder = phase2_model.distilbert
        self.classifier_head = phase2_model.classifier
        self.temperature = temperature

        if freeze_base:
            for param in self.encoder.parameters():
                param.requires_grad = False
            print("✓ Base encoder frozen")
        else:
            print("✓ Base encoder unfrozen (full fine-tuning)")

    def mean_pooling(self, token_embeddings, attention_mask):
        """Mean pooling with attention mask"""
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    def encode(self, input_ids, attention_mask):
        """Encode sentence to embedding"""
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = self.mean_pooling(outputs.last_hidden_state, attention_mask)
        embeddings = F.normalize(embeddings, p=2, dim=1)
        return embeddings

    def forward(self, sent1_ids, sent1_mask, sent2_ids, sent2_mask):
        """Forward pass for contrastive learning"""
        emb1 = self.encode(sent1_ids, sent1_mask)
        emb2 = self.encode(sent2_ids, sent2_mask)
        return emb1, emb2

    def compute_similarity(self, emb1, emb2):
        """Compute cosine similarity"""
        return F.cosine_similarity(emb1, emb2)

In [10]:
class InfoNCELoss(nn.Module):
    """InfoNCE Contrastive Loss"""

    def __init__(self, temperature=0.05):
        super().__init__()
        self.temperature = temperature

    def forward(self, emb1, emb2, labels):
        batch_size = emb1.size(0)

        similarity_matrix = torch.matmul(emb1, emb2.T) / self.temperature

        labels = labels.float()

        loss = 0
        for i in range(batch_size):
            if labels[i] == 1:  # Positive pair
                pos_sim = similarity_matrix[i, i]
                all_sims = torch.exp(similarity_matrix[i])
                loss += -torch.log(torch.exp(pos_sim) / all_sims.sum())
            else:  # Negative pair
                neg_sim = similarity_matrix[i, i]
                loss += torch.exp(neg_sim)

        return loss / batch_size

In [11]:
class ContrastiveHardNegativeMiner:
    """Mine hard negative pairs using Phase 2 model uncertainty"""

    def __init__(self, phase2_model, tokenizer, device):
        self.phase2_model = phase2_model
        self.tokenizer = tokenizer
        self.device = device

    def get_uncertainty_score(self, sentence):
        """Get Phase 2 model's uncertainty"""
        self.phase2_model.eval()

        inputs = self.tokenizer(
            sentence,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=128
        ).to(self.device)

        with torch.no_grad():
            outputs = self.phase2_model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)
            confidence = probs.max().item()

        return 1.0 - confidence

    def mine_hard_negatives(self, pairs, top_k=50):
        """Mine hard negative pairs"""
        print("\n[Hard Negative Mining] Finding uncertain pairs...")

        hard_negatives = []

        for pair in tqdm(pairs):
            if pair['label'] == 0:  # Only negative pairs
                unc1 = self.get_uncertainty_score(pair['sentence1'])
                unc2 = self.get_uncertainty_score(pair['sentence2'])

                avg_uncertainty = (unc1 + unc2) / 2

                pair['uncertainty'] = avg_uncertainty
                hard_negatives.append(pair)

        hard_negatives.sort(key=lambda x: x['uncertainty'], reverse=True)

        print(f"✓ Found {len(hard_negatives)} candidate hard negatives")
        print(f"✓ Selecting top {top_k} most uncertain pairs")

        return hard_negatives[:top_k]

In [12]:
class ContrastivePairDataset(Dataset):
    """Dataset for contrastive sentence pairs"""

    def __init__(self, pairs, tokenizer, max_length=128):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        pair = self.pairs[idx]

        sent1_enc = self.tokenizer(
            pair['sentence1'],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        sent2_enc = self.tokenizer(
            pair['sentence2'],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'sent1_ids': sent1_enc['input_ids'].flatten(),
            'sent1_mask': sent1_enc['attention_mask'].flatten(),
            'sent2_ids': sent2_enc['input_ids'].flatten(),
            'sent2_mask': sent2_enc['attention_mask'].flatten(),
            'label': torch.tensor(pair['label'], dtype=torch.long)
        }


def train_contrastive_epoch(model, dataloader, optimizer, scheduler, loss_fn, device):
    """Train one epoch of contrastive learning"""
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc="Contrastive Training"):
        sent1_ids = batch['sent1_ids'].to(device)
        sent1_mask = batch['sent1_mask'].to(device)
        sent2_ids = batch['sent2_ids'].to(device)
        sent2_mask = batch['sent2_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()

        emb1, emb2 = model(sent1_ids, sent1_mask, sent2_ids, sent2_mask)
        loss = loss_fn(emb1, emb2, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


def evaluate_contrastive(model, dataloader, device):
    """Evaluate contrastive model"""
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            sent1_ids = batch['sent1_ids'].to(device)
            sent1_mask = batch['sent1_mask'].to(device)
            sent2_ids = batch['sent2_ids'].to(device)
            sent2_mask = batch['sent2_mask'].to(device)
            labels = batch['label'].to(device)

            emb1, emb2 = model(sent1_ids, sent1_mask, sent2_ids, sent2_mask)
            similarity = model.compute_similarity(emb1, emb2)

            predictions = (similarity > 0.5).long()

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    return correct / total if total > 0 else 0

In [13]:
# ============================================================================
# MAIN INTEGRATED PIPELINE
# ============================================================================

def run_experiment_2(device):
    """Run Experiment 2: Adversarial Classifier"""

    print("\n" + "="*80)
    print("EXPERIMENT 2: ADVERSARIAL CLASSIFIER WITH HARD NEGATIVE MINING")
    print("="*80)

    # Configuration
    batch_size = 16
    num_epochs = 8
    learning_rate = 2e-5

    # Initialize
    print("\n[1/6] Initializing components...")
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased',
        num_labels=2
    ).to(device)

    detector = LogicalSentenceDetector()
    generator = AdversarialGenerator()

    # Create dataset
    print("\n[2/6] Creating expanded dataset...")
    initial_data, logical_sentences = create_expanded_dataset()

    # Augment
    print("\n[3/6] Generating adversarial examples...")
    augmented_data, perturbed_map = augment_with_multi_perturbations(
        initial_data,
        generator,
        rounds=[1, 2, 3],
        aug_per_round=0.8
    )

    # Split
    train_data, val_data = train_test_split(augmented_data, test_size=0.2, random_state=42)

    train_dataset = LogicDataset(train_data, tokenizer)
    val_dataset = LogicDataset(val_data, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Optimizer
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    # Train
    print("\n[4/6] Training with hard negative mining...")
    best_val_acc = 0
    hard_negative_miner = HardNegativeMiner(model, tokenizer, device)

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-" * 80)

        train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler, device)
        val_loss, val_acc = evaluate(model, val_loader, device)

        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_logic_classifier.pt')
            print(f"✓ Best model saved!")

        # Hard negative mining every 2 epochs
        if (epoch + 1) % 2 == 0 and epoch < num_epochs - 1:
            print("\n  >> Running hard negative mining...")
            hard_examples = hard_negative_miner.find_hard_negatives(train_data, threshold=0.3)

            if len(hard_examples) > 10:
                hard_augmented = hard_negative_miner.augment_hard_negatives(
                    hard_examples, generator, multiplier=2
                )

                train_data.extend(hard_augmented)
                train_dataset = LogicDataset(train_data, tokenizer)
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

                print(f"  ✓ Added {len(hard_augmented)} hard negative augmentations")

    print(f"\n{'='*80}")
    print(f"EXPERIMENT 2 COMPLETE!")
    print(f"Best validation accuracy: {best_val_acc:.4f}")
    print(f"Model saved: best_logic_classifier.pt")
    print(f"{'='*80}")
    # Save data for Experiment 3
    with open('logical_sentences.json', 'w') as f:
        json.dump(logical_sentences, f, indent=2)

    with open('perturbed_sentences.json', 'w') as f:
        json.dump(perturbed_map, f, indent=2)

    print("\n✓ Saved logical_sentences.json and perturbed_sentences.json for Experiment 3")

    return model, tokenizer, logical_sentences, perturbed_map, best_val_acc


In [14]:
def run_experiment_3(phase2_model, tokenizer, logical_sentences, perturbed_map, device):
    """Run Experiment 3: Contrastive Learning"""

    print("\n" + "="*80)
    print("EXPERIMENT 3: CONTRASTIVE LOGICAL PRETRAINING")
    print("="*80)

    # Configuration
    batch_size = 32
    num_epochs = 3
    learning_rate = 2e-5
    use_hard_mining = True

    # Construct contrastive pairs
    pairs = construct_contrastive_dataset(logical_sentences, perturbed_map)

    # Hard negative mining
    if use_hard_mining:
        print("\n[Optional] Mining hard negatives from Phase 2 uncertainty...")
        miner = ContrastiveHardNegativeMiner(phase2_model, tokenizer, device)
        hard_negatives = miner.mine_hard_negatives(pairs, top_k=50)

        pairs.extend(hard_negatives * 2)  # 2x weight
        random.shuffle(pairs)
        print(f"✓ Dataset now contains {len(pairs)} pairs (including hard negatives)")

    # Split
    train_pairs, val_pairs = train_test_split(pairs, test_size=0.2, random_state=42)
    print(f"\n✓ Split: {len(train_pairs)} train, {len(val_pairs)} validation")

    # Dataloaders
    train_dataset = ContrastivePairDataset(train_pairs, tokenizer)
    val_dataset = ContrastivePairDataset(val_pairs, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Create contrastive model
    print("\n[Model Setup] Wrapping Phase 2 model with contrastive objective...")
    contrastive_model = ContrastiveWrapper(
        phase2_model,
        temperature=0.05,
        freeze_base=False
    ).to(device)

    # Optimizer
    loss_fn = InfoNCELoss(temperature=0.05)
    optimizer = AdamW(contrastive_model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(0.1 * total_steps),
        num_training_steps=total_steps
    )

    # Train
    print("\n[Training] Lightweight contrastive training (2-3 epochs)...")
    best_val_acc = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-" * 80)

        train_loss = train_contrastive_epoch(
            contrastive_model, train_loader, optimizer, scheduler, loss_fn, device
        )
        val_acc = evaluate_contrastive(contrastive_model, val_loader, device)

        print(f"Train Loss: {train_loss:.4f} | Val Accuracy: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(contrastive_model.state_dict(), 'contrastive_model_exp3.pt')
            print(f"✓ Best contrastive model saved!")

    print(f"\n{'='*80}")
    print(f"EXPERIMENT 3 COMPLETE!")
    print(f"Best validation accuracy: {best_val_acc:.4f}")
    print(f"Model saved: contrastive_model_exp3.pt")
    print(f"{'='*80}")

    # Test
    print("\n[Testing] Sample predictions...")
    print("-" * 80)

    contrastive_model.eval()
    test_pairs = [
        ("If it rains, the ground will be wet.", "When it rains, the ground becomes wet.", True),
        ("Because he studied, he passed.", "His studying led to passing the exam.", True),
        ("If it rains, the ground will be wet.", "The cat is sleeping.", False),
        ("Because he studied, he passed.", "The sky is blue.", False),
    ]

    with torch.no_grad():
        for sent1, sent2, is_related in test_pairs:
            enc1 = tokenizer(sent1, return_tensors='pt', padding=True, truncation=True).to(device)
            enc2 = tokenizer(sent2, return_tensors='pt', padding=True, truncation=True).to(device)

            emb1 = contrastive_model.encode(enc1['input_ids'], enc1['attention_mask'])
            emb2 = contrastive_model.encode(enc2['input_ids'], enc2['attention_mask'])

            similarity = contrastive_model.compute_similarity(emb1, emb2).item()
            relation = "RELATED" if is_related else "UNRELATED"

            print(f"\n[{relation}]")
            print(f"  Sent 1: '{sent1}'")
            print(f"  Sent 2: '{sent2}'")
            print(f"  → Similarity: {similarity:.4f}")

    # Save results
    results = {
        'experiment': 'contrastive_extension',
        'num_epochs': num_epochs,
        'best_val_accuracy': best_val_acc,
        'dataset_size': len(pairs)
    }

    with open('experiment3_results.json', 'w') as f:
        json.dump(results, f, indent=2)

    print("\n✓ Results saved to: experiment3_results.json")

    return contrastive_model, best_val_acc

In [15]:
def main():
    """Main pipeline: Run Experiments 2 and 3"""

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print("="*80)
    print("INTEGRATED PIPELINE: EXPERIMENTS 2 → 3")
    print("="*80)
    print(f"Device: {device}")
    print(f"Running both experiments sequentially...")

    # Run Experiment 2
    phase2_model, tokenizer, logical_sentences, perturbed_map, exp2_acc = run_experiment_2(device)

    # Run Experiment 3
    contrastive_model, exp3_acc = run_experiment_3(
        phase2_model, tokenizer, logical_sentences, perturbed_map, device
    )

    # Final summary
    print("\n" + "="*80)
    print("PIPELINE COMPLETE - FINAL SUMMARY")
    print("="*80)
    print(f"\nExperiment 2 (Adversarial):")
    print(f"  Best Val Accuracy: {exp2_acc:.4f}")
    print(f"  Model: best_logic_classifier.pt")

    print(f"\nExperiment 3 (Contrastive):")
    print(f"  Best Val Accuracy: {exp3_acc:.4f}")
    print(f"  Model: contrastive_model_exp3.pt")

    print(f"\n✓ All files saved successfully!")
    print("="*80)

    return phase2_model, contrastive_model


if __name__ == "__main__":
    phase2_model, contrastive_model = main()

INTEGRATED PIPELINE: EXPERIMENTS 2 → 3
Device: cuda
Running both experiments sequentially...

EXPERIMENT 2: ADVERSARIAL CLASSIFIER WITH HARD NEGATIVE MINING

[1/6] Initializing components...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Train Loss: 3678221.3276 | Val Accuracy: 0.5906
✓ Best contrastive model saved!

Epoch 2/3
--------------------------------------------------------------------------------


Evaluating: 100%|██████████| 4/4 [00:00<00:00,  4.52it/s]


Train Loss: 67071.6725 | Val Accuracy: 0.8189
✓ Best contrastive model saved!

Epoch 3/3
--------------------------------------------------------------------------------


Evaluating: 100%|██████████| 4/4 [00:00<00:00,  4.57it/s]


Train Loss: 25222.8713 | Val Accuracy: 0.8346
✓ Best contrastive model saved!

EXPERIMENT 3 COMPLETE!
Best validation accuracy: 0.8346
Model saved: contrastive_model_exp3.pt

[Testing] Sample predictions...
--------------------------------------------------------------------------------

[RELATED]
  Sent 1: 'If it rains, the ground will be wet.'
  Sent 2: 'When it rains, the ground becomes wet.'
  → Similarity: 0.8437

[RELATED]
  Sent 1: 'Because he studied, he passed.'
  Sent 2: 'His studying led to passing the exam.'
  → Similarity: 0.3112

[UNRELATED]
  Sent 1: 'If it rains, the ground will be wet.'
  Sent 2: 'The cat is sleeping.'
  → Similarity: -0.1324

[UNRELATED]
  Sent 1: 'Because he studied, he passed.'
  Sent 2: 'The sky is blue.'
  → Similarity: 0.0126

✓ Results saved to: experiment3_results.json

PIPELINE COMPLETE - FINAL SUMMARY

Experiment 2 (Adversarial):
  Best Val Accuracy: 0.9732
  Model: best_logic_classifier.pt

Experiment 3 (Contrastive):
  Best Val Accuracy: 0.